In [1]:
import os
import pickle
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
from sklearn.pipeline import make_pipeline

In [3]:
import mlflow

#MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
#mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
#mlflow.set_experiment("green-taxi-duration-s3")

# For storing artifacts in s3, tracking uri can be localhost, but launch the server from terminal
# with the proper s3 artifact root, for example:
# mlflow server --backend-store-uri=sqlite:///mlflow.db --default-artifact-root=s3://mlops-zoomcamp-taxi-data/models/


<Experiment: artifact_location='s3://mlops-zoomcamp-taxi-data/models/2', creation_time=1689387407819, experiment_id='2', last_update_time=1689387407819, lifecycle_stage='active', name='green-taxi-duration-s3', tags={}>

In [4]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [11]:
import boto3

s3 = boto3.client('s3')

In [16]:
filename_train = 'green_tripdata_2022-01.parquet'
filename_val = 'green_tripdata_2022-02.parquet'

s3.download_file('mlops-zoomcamp-taxi-data', 'data/green_tripdata_2022-01.parquet', filename_train)
s3.download_file('mlops-zoomcamp-taxi-data', 'data/green_tripdata_2022-02.parquet', filename_val)

df_train = read_dataframe(filename_train)
df_val = read_dataframe(filename_val)

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [17]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.101201727016495
